# 2024년 일 단위 위험지수 실험 노트북 (Daily Risk Experiments)

- 목적  
  - `data/mart/monthly_predict_vs_actual.csv`에서 월 단위 예측값(`y_hat`)을 불러와  
    2024년 1월 1일 ~ 12월 31일까지 일 단위 호흡기 질환 위험지수(risk_score, risk_level)를 생성하는 실험이다.  
  - `backend/modeling/daily_risk_index/monthly_tree_and_daily_risk.py`에 구현된  
    `get_daily_risk()` 함수를 노트북 환경에서 호출하여,  
    날짜별 · 시도별 일일 위험지수 CSV를 생성하고 결과 분포를 확인하는 것을 목표로 한다.

- 사용 데이터 및 코드  
  - 입력 데이터  
    - `data/mart/merged_monthly.csv` : 월 단위 통합 데이터 (건수, 인구, 대기질, 기상 등)  
    - `data/mart/monthly_predict_vs_actual.csv` : 월 단위 회귀트리 예측 결과 (`y_hat`, `error_pct` 등)  
  - 사용 모듈  
    - `backend/modeling/daily_risk_index/monthly_tree_and_daily_risk.py`  
      - `train_monthly_tree()` : 월 단위 회귀트리 학습 및 평가 (이미 실행된 상태라고 가정)  
      - `get_daily_risk(date_str, ...)` : 특정 날짜에 대한 일 단위 위험지수 계산

- 이 노트북에서 수행하는 주요 실험  
  1. 2024년 날짜 범위(2024-01-01 ~ 2024-12-31) 생성  
  2. 각 날짜에 대해 `get_daily_risk()`를 호출하여 일 단위 위험지수 계산  
  3. 시도별로 2024년 일일 위험지수 CSV 저장  
     - 예: `data/model/daily_risk_index/daily_risk_2024_서울.csv`, `daily_risk_2024_제주.csv` 등  
  4. 2024년 전체 기간에 대한 분포 확인  
     - 시도별 평균 risk_score, High/Very High 비율, 월별 경향 등 단순 요약

- 주의 사항  
  - 이 노트북은 **“2023년까지 학습한 월 단위 모델을 이용해 2024년 위험도를 근사 추정하는 실험용(EDA) 노트북”** 이다.  
  - `get_daily_risk()`의 환경 가중치(SEASON_WEIGHTS, pm25 기반 weight 등)는  
    개념적인 위험도 비교용으로 설계됐으며,  
    **실제 역학적 위험도를 보장하지 않는다.**  
  - 최종 리포트 및 발표에 사용되는 공식 결과는  
    필요 시 별도의 검증·튜닝 과정을 거쳐 정리해야 한다.  
  - 이 노트북은 실험 기록 및 재현 가능한 워크플로우 공유를 위한 용도이며,  
    코드/결과는 언제든 수정·보완될 수 있다.

In [2]:
from pathlib import Path
import sys
import pandas as pd

# 1) 프로젝트 루트 자동 탐지 (notebooks/ 안에서 실행된다고 가정)
cwd = Path.cwd()
print("현재 작업 디렉터리:", cwd)

# 만약 현재 폴더 이름이 'notebooks' 이면 한 단계 위가 루트
if cwd.name == "notebooks":
    PROJECT_ROOT = cwd.parent
else:
    # 혹시 루트에서 바로 열었을 때도 동작하도록
    PROJECT_ROOT = cwd

print("PROJECT_ROOT:", PROJECT_ROOT)

# monthly_tree_and_daily_risk.py 가 있는 폴더 경로
daily_risk_dir = PROJECT_ROOT / "backend" / "modeling" / "daily_risk_index"
print("daily_risk_dir:", daily_risk_dir)

# 모듈 import 경로 추가
sys.path.append(str(daily_risk_dir))

from monthly_tree_and_daily_risk import (
    train_monthly_tree,
    get_daily_risk,
    load_monthly_predictions,
    OUTPUT_DIR,
    MONTHLY_PRED_PATH,
)

print("OUTPUT_DIR:", OUTPUT_DIR)
print("MONTHLY_PRED_PATH:", MONTHLY_PRED_PATH)

현재 작업 디렉터리: c:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\notebooks
PROJECT_ROOT: c:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab
daily_risk_dir: c:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\backend\modeling\daily_risk_index
OUTPUT_DIR: C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index
MONTHLY_PRED_PATH: C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\mart\monthly_predict_vs_actual.csv


In [3]:
# 2) 월 단위 회귀트리 다시 학습해서 CSV/그래프 재생성 (선택)
monthly_df = train_monthly_tree()
monthly_df.head()

=== 1) 데이터 로드 ===
입력 데이터 shape: (1428, 17)
Train size: 949
Test size : 194

=== 하이퍼파라미터 탐색 결과 (상위 몇 개) ===
   max_depth  min_samples_leaf   MAE_train  RMSE_train    MAE_test   RMSE_test
0          3                10  319.097194  425.134986  322.847999  408.569251
1          3                20  319.970978  425.598784  324.119462  409.895832
2          5                20  256.867881  355.591005  325.334405  411.059771
3          4                20  281.389397  382.272782  322.091686  414.405826
4          4                10  285.557362  379.994779  321.806408  418.982986
하이퍼파라미터 결과 저장: C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\monthly_dt_results.csv

=== 선택된 최종 파라미터 ===
max_depth=3, min_samples_leaf=10
월 단위 예측 결과 저장: C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\mart\monthly_predict_vs_actual.csv
시도별 오차 요약 저장: C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_

,year,month,ym,sido_name,resp_rate_total,y_hat,error_abs,error_pct
0,2018,2,2018-02,강원,2809.361174,2863.316597,53.955423,1.920558
1,2018,3,2018-03,강원,2710.535575,2863.316597,152.781022,5.636562
2,2018,4,2018-04,강원,2737.839804,2863.316597,125.476793,4.583058
3,2018,5,2018-05,강원,2570.972069,2863.316597,292.344528,11.370973
4,2018,6,2018-06,강원,1919.159423,1794.584575,124.574848,6.491115


In [4]:
# 3) 월 단위 예측 CSV 로드
monthly_preds = load_monthly_predictions(MONTHLY_PRED_PATH)
monthly_preds.head()

,year,month,ym,sido_name,resp_rate_total,y_hat,error_abs,error_pct
0,2018,2,2018-02,강원,2809.361174,2863.316597,53.955423,1.920558
1,2018,3,2018-03,강원,2710.535575,2863.316597,152.781022,5.636562
2,2018,4,2018-04,강원,2737.839804,2863.316597,125.476793,4.583058
3,2018,5,2018-05,강원,2570.972069,2863.316597,292.344528,11.370973
4,2018,6,2018-06,강원,1919.159423,1794.584575,124.574848,6.491115


In [5]:
# 4) 2024-01-01 ~ 2024-12-01 까지 월별로 일일 위험지수 계산
all_rows_2024 = []

for m in range(1, 13):
    date_str = f"2024-{m:02d}-01"
    print(f"→ {date_str} 계산 중...")

    df_daily = get_daily_risk(
        date_str,
        monthly_preds_df=monthly_preds,  # 3번 셀에서 로드한 월 예측 사용
        # env_obs_df=None, ref_df=None  # 지금은 환경관측 안 넣을 거면 기본값(None) 유지
    )
    all_rows_2024.append(df_daily)

df_2024 = pd.concat(all_rows_2024, ignore_index=True)
df_2024.head()

→ 2024-01-01 계산 중...
→ 2024-02-01 계산 중...
→ 2024-03-01 계산 중...
→ 2024-04-01 계산 중...
→ 2024-05-01 계산 중...
→ 2024-06-01 계산 중...
→ 2024-07-01 계산 중...
→ 2024-08-01 계산 중...
→ 2024-09-01 계산 중...
→ 2024-10-01 계산 중...
→ 2024-11-01 계산 중...
→ 2024-12-01 계산 중...


,date,year,month,ym,sido_name,y_hat,base_daily,w_today,y_day_hat,risk_score,risk_level
0,2024-01-01,2024,1,2024-01,강원,2863.316597,92.365052,1.2,110.838062,100.0,Very High
1,2024-01-01,2024,1,2024-01,경기,2863.316597,92.365052,1.2,110.838062,100.0,Very High
2,2024-01-01,2024,1,2024-01,경남,2863.316597,92.365052,1.2,110.838062,100.0,Very High
3,2024-01-01,2024,1,2024-01,경북,2863.316597,92.365052,1.2,110.838062,100.0,Very High
4,2024-01-01,2024,1,2024-01,광주,2863.316597,92.365052,1.2,110.838062,100.0,Very High


In [6]:
# 5) 2024 전체 시도 일일 위험지수 CSV 저장
out_all = OUTPUT_DIR / "daily_risk_2024_all_regions.csv"
out_all.parent.mkdir(parents=True, exist_ok=True)

df_2024.to_csv(out_all, index=False, encoding="utf-8-sig")
out_all

WindowsPath('C:/dev/university/2025_빅데이터기초및실습/TeamProject/GitHubRepo/breath-datalab/data/model/daily_risk_index/daily_risk_2024_all_regions.csv')

In [7]:
# 6) 시도별(광역시·도별)로 CSV 분리 저장
for sido, g in df_2024.groupby("sido_name"):
    safe_name = sido.replace(" ", "")
    out_sido = OUTPUT_DIR / f"daily_risk_2024_{safe_name}.csv"
    g.to_csv(out_sido, index=False, encoding="utf-8-sig")
    print(f"{sido} → {out_sido}")

강원 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_강원.csv
경기 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_경기.csv
경남 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_경남.csv
경북 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_경북.csv
광주 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_광주.csv
대구 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_대구.csv
대전 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_risk_2024_대전.csv
부산 → C:\dev\university\2025_빅데이터기초및실습\TeamProject\GitHubRepo\breath-datalab\data\model\daily_risk_index\daily_r

In [8]:
# 7) 위험등급 분포 체크 (선택)
df_2024["risk_level"].value_counts()

# 7-1) 월별 평균 위험점수 (선택)
monthly_mean_score = df_2024.groupby(["month"])["risk_score"].mean()
monthly_mean_score

# 7-2) 시도별 평균 위험점수 (선택)
region_mean_score = (
    df_2024.groupby(["sido_name"])["risk_score"].mean().sort_values(ascending=False)
)
region_mean_score

sido_name
전남    75.322389
강원    73.906329
경기    65.453287
충북    63.830989
충남    63.020477
제주    63.016251
전북    62.457112
부산    60.866507
경북    60.866507
인천    60.866507
경남    60.866507
서울    60.866507
대전    60.456330
울산    58.258907
광주    51.963227
대구    50.049182
세종    41.497726
Name: risk_score, dtype: float64